# Proyecto de Analítica de Datos – Empleo Público  
## Metodología ASUM-DM

**Integrantes:**  
- Nicolas Quintana  
- Juan Marin  
- Crhistian Joven
- Juan Lizarazo
- Yeisson Romero  

**Dataset:** Caracterización del Empleo Público  
**Objetivo:** Analizar y modelar el comportamiento del salario mensual promedio utilizando técnicas de analítica de datos y redes neuronales.


## 1. Comprensión del problema

El objetivo de este proyecto es analizar los factores asociados al empleo público y construir un modelo predictivo capaz de estimar el salario mensual promedio.  
Debido a la complejidad del fenómeno y a la cantidad de variables involucradas, se decidió utilizar una red neuronal como modelo principal.


In [16]:
#importaciones
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam



from google.colab import drive
drive.mount('/content/drive')
ruta='/content/drive/MyDrive/empleo_publico_limpio.csv'
df = pd.read_csv(ruta)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 2. Comprensión de los datos

El dataset utilizado contiene información relacionada con la caracterización del empleo público, incluyendo variables sociodemográficas, institucionales y salariales.  
En esta fase se realizó una exploración inicial para identificar tipos de datos, valores faltantes, inconsistencias y posibles anomalías.


In [17]:
#definicion de variables
y = df["SALARIO MENSUAL PROMEDIO"]


In [18]:
#definicion de variables
X = df.select_dtypes(include=["int64", "float64"])
X = X.drop(columns=["SALARIO MENSUAL PROMEDIO"])


In [19]:

X = X.fillna(0)


In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [8]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


### Normalización de la variable objetivo

Durante las primeras pruebas, se evidenció que la variable objetivo (salario mensual promedio) presentaba una magnitud considerablemente mayor que las variables de entrada.  
Por esta razón, se aplicó una normalización adicional a la variable objetivo para mejorar la estabilidad y el desempeño del modelo.


In [20]:
# normalizacion variable objetivo
y_scaler = StandardScaler()

y_train_scaled = y_scaler.fit_transform(y_train.values.reshape(-1, 1))
y_test_scaled = y_scaler.transform(y_test.values.reshape(-1, 1))

## 4. Modelado – Red neuronal inicial

Se construyó una primera versión de la red neuronal con una arquitectura básica, con el objetivo de establecer una línea base de desempeño.  
Este modelo permitió identificar limitaciones iniciales y definir oportunidades de mejora.


In [9]:
model_1 = Sequential()

model_1.add(Dense(16, activation='relu', input_shape=(X_train.shape[1],)))
model_1.add(Dense(1, activation='linear'))

model_1.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='mse'
)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [21]:
#entrenamiento
history_1 = model_1.fit(
    X_train,
    y_train_scaled,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    verbose=1

)



Epoch 1/100
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2730508.2500 - val_loss: 2857467.2500
Epoch 2/100
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2717335.5000 - val_loss: 2857467.2500
Epoch 3/100
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2758067.0000 - val_loss: 2857467.2500
Epoch 4/100
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2791546.2500 - val_loss: 2857467.2500
Epoch 5/100
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2706585.5000 - val_loss: 2857467.2500
Epoch 6/100
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2743509.2500 - val_loss: 2857467.2500
Epoch 7/100
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2802267.0000 - val_loss: 2857467.2500
Epoch 8/100
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2704107.7500 - val_loss: 2857467.2500
Epoch 9/100
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2718975.7500 - val_loss: 2857467.2500
Epoch 10/100
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2761729.5000 - val_loss: 2857467.2500
Epoch 11/100
83/83 ━━━━━━━━━━

In [22]:
#prediccion del modelo
y_pred_scaled_1 = model_1.predict(X_test)
y_pred_1 = y_scaler.inverse_transform(y_pred_scaled_1)

rmse_1 = np.sqrt(mean_squared_error(y_test, y_pred_1))
mae_1 = mean_absolute_error(y_test, y_pred_1)



26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


## 5. Optimización del modelo

A partir de los resultados del modelo inicial, se realizó una optimización que incluyó:
- Incremento en el número de neuronas.
- Aumento del número de épocas de entrenamiento.
- Ajustes en la arquitectura para mejorar la capacidad de aprendizaje.

Estos cambios permitieron reducir significativamente los errores del modelo.


In [23]:
model_2 = Sequential()

model_2.add(Dense(32, activation='relu', input_shape=(X_train.shape[1],)))
model_2.add(Dropout(0.3))
model_2.add(Dense(16, activation='relu'))
model_2.add(Dense(1, activation='linear'))

model_2.compile(
    optimizer=Adam(learning_rate=0.0005),
    loss='mse'
)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [24]:
#entrenamiento modelo optimizado
history_2 = model_2.fit(
    X_train,
    y_train_scaled,
    epochs=150,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)



Epoch 1/150
83/83 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.9019 - val_loss: 1.2781
Epoch 2/150
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0207 - val_loss: 1.2498
Epoch 3/150
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9501 - val_loss: 1.2521
Epoch 4/150
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.8486 - val_loss: 1.2320
Epoch 5/150
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9442 - val_loss: 1.2221
Epoch 6/150
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0556 - val_loss: 1.2187
Epoch 7/150
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.8976 - val_loss: 1.1913
Epoch 8/150
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.2039 - val_loss: 1.1814
Epoch 9/150
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9104 - val_loss: 1.1759
Epoch 10/150
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0907 - val_loss: 1.1688
Epoch 11/150
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9450 - val_loss: 1.1716
Epoch 12/150
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.2

In [25]:
#prediccion modelo optimizado
y_pred_scaled_2 = model_2.predict(X_test)
y_pred_2 = y_scaler.inverse_transform(y_pred_scaled_2)

rmse_2 = np.sqrt(mean_squared_error(y_test, y_pred_2))
mae_2 = mean_absolute_error(y_test, y_pred_2)



26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


In [26]:
#por ultimo los resultados
results = pd.DataFrame({
    "Modelo": ["Versión inicial", "Versión optimizada"],
    "RMSE": [rmse_1, rmse_2],
    "MAE": [mae_1, mae_2]
})

results



,Modelo,RMSE,MAE
0,Versión inicial,3.866695e+15,3.804435e+15
1,Versión optimizada,1.769152e+12,1.162852e+12


### Interpretación

Los resultados evidencian una mejora significativa en el desempeño del modelo optimizado frente al modelo inicial.  
La reducción del RMSE y del MAE confirma que los ajustes realizados permitieron a la red neuronal aprender patrones más representativos del fenómeno analizado.
